In [ ]:
import os
import sys
from bs4 import BeautifulSoup
import urllib.request as ul
import requests as r
import csv
import pandas as pd
import re
from selenium import webdriver
from parsel import Selector 
from collections import defaultdict
import time

In [ ]:
#called function
def getHouseLinks():
    all_links=[]
    for i in range(100,200):
 
        driver = webdriver.Chrome()
        url = ('https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page=' + str(i) + '&orderBy=relevance')
        driver.get(url)
    
        #print("Search Criteria :"  + url)
        soup = BeautifulSoup(driver.page_source, "html.parser")    
               
        rawHouseLinks = soup.find_all("a", {"class":"card__title-link"}) 

        for item in rawHouseLinks:
            try:
                
                link = item.get('href') 
                #print(link)                      
                all_links.append(link)

            except AttributeError:
                rawHouseLinks != 'data' 

        driver.close()
        

    return all_links
#end of function


In [ ]:
def getPropertyType(url):
    propType = ''
    if url.find('apartment')>0:
        propType = 'Apartment'
    elif url.find('house')>0:
        propType = 'House'
    elif url.find('mansion')>0:
        propType = 'Mansion'
    elif url.find('chalet')>0:
        propType = 'Chalet'
    elif url.find('castle')>0:
        propType = 'Castle'
    elif url.find('farmhouse')>0:
        propType = 'Farmhouse'
    elif url.find('exceptional property')>0:
        propType = 'Exceptional property'
    elif url.find('bungalow')>0:
        propType = 'Bungalow'
    elif url.find('town-house')>0:
        propType = 'Town-house'
    else:
        propType = 'Mixed-use building'
    
    return propType

In [ ]:
def GetFieldsOfInterest(id,locality,sale_rent, zipcode,prop_type):
    fields = {   'ID':id, 
        'Locality': locality,
        'Sale_rent': sale_rent,
        #'Monthly rental price':0,
        'Zipcode' : zipcode,
        'Building condition': False,
        'Price': 0,
        'Kitchen type':False,
        'Living area': 0,
        'Furnished': False,
        'Surface of the plot': 0,
        'Number of frontages': 0,
        'Swimming pool': False,
        'Bedrooms': 0,
        'Garden' : False,
        'Garden surface' : 0,
        'Terrace':False,
        'Terrace surface': 0,
        'Type of property' : prop_type   
                            
        }
    return fields

In [ ]:
#cleaning the data
def getData(hlinks):
  
  houseInfo = [] 
  for item in hlinks:
    try:   
      driver = webdriver.Chrome()
      driver.get(item)
     
      soup = BeautifulSoup(driver.page_source, "html.parser")   
          
      table = soup.find_all("table", {"class":"classified-table"})    
      rows=[]
      for t in table:     
        trs = t.findAll("tr")
        for tr in trs:
          rows.append(tr)
      
      driver.close()
      id = item.rsplit('/')[-1]
      zipCode = item.rsplit('/')[-2]
      locality = item.rsplit('/')[-3]
      saleRent = item.rsplit('/')[-4]
    
      url = str(item)  
      propType = getPropertyType(url)
       
      #data={}
      fieldsOfInterest = GetFieldsOfInterest(id,locality,saleRent,zipCode,propType)
      

      for item in rows:         
        try:      
          if len(item.findAll("th", {"class":"classified-table__header"})) > 0: #   check if the html found then only read the Text 
            key  = str(item.find("th", {"class":"classified-table__header"}).getText()).replace("  "," ").strip(" ,\r\n\t")          
            v1 = str(item.find("td", {"class":"classified-table__data"}).getText()).replace("  "," ").strip(" ,\r\n\t") 
            value = " ".join(v1.split()) # this we are doing to remove the new line from 1600 M2
            #print(str(key)+ ":::" + str(value))
            if(fieldsOfInterest.__contains__(key)): 
              
              fieldsOfInterest[key] =  value
              
              # All custom logic goes here : 
              #Sale price
              if key == 'Price' :           
                #p= value.replace(',','').split() 
                valuewithoutcomma= value.replace(',','')
                p1= re.findall("\d+", valuewithoutcomma)    
                fieldsOfInterest['Price'] = int(p1[0])
              #swimming pool: changed boolen value
              if key == 'Swimming pool':
                if value == 'Yes':
                  fieldsOfInterest[key] = True
                else:
                  fieldsOfInterest[key] = False
              # Furnished    
              if key == 'Furnished':
                if value == 'Yes':
                  fieldsOfInterest[key] = True
                else:
                  fieldsOfInterest[key] = False
              # Terrace    
              if key == 'Terrace':
                if value == 'Yes' :#or 'Terrace surface'>0:
                  fieldsOfInterest[key] = True
                else:
                  fieldsOfInterest[key] = False
                  
              if key == 'Terrace surface':
                fieldsOfInterest["Terrace"] = True          
                                
              # Garden    
              if key == 'Garden':
                if value == 'Yes':
                  fieldsOfInterest[key] = True
                else:
                  fieldsOfInterest[key] = False

                  
              if key == 'Garden surface':
                fieldsOfInterest['Garden']=True
  
                
              # Kitchen Type    
              if key == 'Kitchen type':
                if value != 'Not Installed':
                  fieldsOfInterest[key] = True
                else:
                  fieldsOfInterest[key] = False
              #for rental price
              #if key == 'Monthly rental price' :           
              #  p= value.replace(',','').split() 
                # valuewithoutcomma= value.replace(',','') 
                #p1= re.findall("\d+", valuewithoutcomma)                   
                #fieldsOfInterest['Monthly rental price'] = int(p1[0])
              #for building condition: Taking only New,renovated and good condition property
              if key == 'Building condition':
                  if value != 'To be done up' or 'To renovate' or 'To restore':
                      fieldsOfInterest[key] = True

              #for Energy Class: Taking only A++,A+,A,B,C
              if key == 'Energy class':
                  if value != 'D' and 'E' and 'F'and 'G':
                      fieldsOfInterest[key] = True
        except:
            print("Warning : Trying to read some out of scope section")  
    
      houseInfo.append(fieldsOfInterest)
      
      
    except:
          print("Warning : Trying to read some out of scope section")  
    
  return houseInfo
          

  
  

In [ ]:
import pandas as pd
from datetime import datetime

def exportToCsv(Data):
    
  now = datetime.now()    
  df = pd.DataFrame.from_dict(Data) 
  fileNameSuffix = now.strftime("%d_%m_%Y_%H_%M_%S")
  fileName = 'Immo_Sale_' + fileNameSuffix + ".csv"
  df.to_csv (fileName,index = False, header=True)

In [ ]:
hlinks = getHouseLinks()

houseinfo = getData(hlinks)

exportToCsv(houseinfo)

